## Drunk driving offense and its traffic consequences vs. alcohol consumption in Poland 2001-2021

### 1. topic description

In [3]:
id = 2
topicName = 'driving-under-influence-pl'
title = """
Drunk driving offense and its traffic consequences vs. alcohol consumption in Poland.
""".replace('\n',' ').strip()
titlePl = """
Przestępstwo jazdy pod wpływem alkoholu i jego skutki drogowe a spożycie alkoholu w Polsce.
""".replace('\n',' ').strip()
country = 'Poland'
countryPl = 'Polska'
startYear = 2001
endYear = 2021
description="""
Statistics of the crime of driving under the influence of alcohol a
nd the consequences of traffic incidents committed under the influence of alcohol (Poland).
Data published by the Polish Police Headquarters.
In addition, a data visualization of the average annual
consumption of alcoholic beverages per capita 
per liter of 100% alcohol (Poland).
Data on alcohol consumption is collected and published by the Polish 
State Agency for Solving Alcohol Problems.
""".replace('\n',' ').strip()
descriptionPl="""
Statystyki przestępstwa jazdy pod wpływem alkoholu 
i skutki zdarzeń drogowych dokonanych pod wpływem alkoholu (Polska). 
Dane publikowane przez polską Komendę Główną Policji.
Dodatkowo została umieszczona wizualizacja danych o średnim rocznym
spożyciu napojów alkoholowych na jednego mieszkańca 
w przeliczeniu na jeden litr alkoholu 100% (Polska).
Dane o spożyciu alkoholu są gromadzone i publikowane przez polską 
Państwową Agencję Rozwiązywania Problemów Alkoholowych.
""".replace('\n',' ').strip()
sourceName = "Komenda Główna Policji"
sourceNamePl =" Police Headquarters (PL)"
sourceLink="https://dane.gov.pl/pl/dataset/2737"
fileName= "dui-pl"
sourceFileExt = 'csv'


In [ ]:
from src.database.insert_topic import insert_topic

insert_topic(
    id,
    topicName,
    title,
    titlePl,
    country,
    countryPl,
    startYear,
    endYear,
    description,
    descriptionPl,
    sourceName,
    sourceNamePl,
    sourceLink,
    fileName,
    sourceFileExt
)

### 2. data exploration

- only second third column is needed
- only first 21 rows are useful

In [5]:
import pandas as pd

original_data_path = '../data/raw/driving-under-influence-pl/dui_pl.csv'

df = pd.read_csv(original_data_path, 
                   skiprows=2,  encoding='iso-8859-1', sep=';')

df.head(25)

,Jednostka podzia³u administracyjnego,Rok,Przestêpstwa stwierdzone,Przestêpstwa wykryte,% wykrycia,Unnamed: 5
0,Polska,2021.0,58 085,58 054,"99,9",NaN
1,Polska,2020.0,52 907,52 861,"99,9",NaN
2,Polska,2019.0,56 207,56 158,"99,9",NaN
3,Polska,2018.0,51 499,51 464,"99,9",NaN
4,Polska,2017.0,55 170,55 089,"99,8",NaN
5,Polska,2016.0,60 443,60 333,"99,8",NaN
6,Polska,2015.0,64 336,64 192,"99,8",NaN
7,Polska,2014.0,73 920,73 787,"99,8",NaN
8,Polska,2013.0,128 064,127 908,"99,9",NaN
9,Polska,2012.0,141 863,141 958,"100,0",NaN


In [6]:
df = df.iloc[:21,[1,2]]

In [7]:
df.iloc[:,1] = pd.to_numeric(df.iloc[:,1].str.replace(' ', ''), errors='coerce').astype(pd.Int64Dtype())

df = df.astype(int)


### 2. Columns names & descriptions

In [8]:
column_names_descriptions = {
    0: ('year', 
        'year', 
        'rok'),
    1: ('numberOfCrimes', 
        'number of crimes found', 
        'liczba stwierdzonych przestępstw'),
}

new_column_names = [t[0] for t in column_names_descriptions.values()]

len(new_column_names)


2

In [9]:
df.columns = new_column_names

In [10]:
df.sort_values(ascending=True, by="year", inplace=True)

In [11]:
df.reset_index(inplace=True, drop=True)
df.head()

,year,numberOfCrimes
0,2001,120113
1,2002,144412
2,2003,150901
3,2004,158543
4,2005,178571


In [12]:
df = df.astype(int)

### 3. create db tables

In [13]:
from src.database.connect_db import connect_db

db = connect_db()

cursor = db.cursor()

In [14]:
MAIN_TABLE_NAME = 'duiPl'

In [15]:
query = f"CREATE TABLE {MAIN_TABLE_NAME} (year INT PRIMARY KEY, "

for column in df.columns[1:]:
    query += f"{column} INT NOT NULL, "


query = query[:-2] + ")"

In [16]:
try:
    cursor.execute(query)
except:
    print('table already created')

In [17]:
data = [tuple(row) for index, row in df.iterrows()]

In [18]:
query = f"""
INSERT INTO {MAIN_TABLE_NAME} ({', '.join(df.columns)}) 
VALUES ({', '.join(['%s']*len(df.columns))});
"""

try:
    cursor.executemany(query, data)
except:
    print('data added already')

In [19]:
cursor.execute(f"""
               SELECT * 
               FROM {MAIN_TABLE_NAME}
               LIMIT 5
               """)
cursor.fetchall()


((2001, 120113),
 (2002, 144412),
 (2003, 150901),
 (2004, 158543),
 (2005, 178571))

In [20]:
from src.database.create_ds_desc_table import create_ds_desc_table


create_ds_desc_table(MAIN_TABLE_NAME)

In [21]:
data = [row for row in column_names_descriptions.values()]

In [22]:
from src.database.insert_into_ds_desc import insert_into_ds_desc


insert_into_ds_desc(MAIN_TABLE_NAME, data)

### 4. upload files to GCP storage

In [23]:
df_desc = pd.DataFrame(data, 
    columns=['column_name', 
    'description', 
    'descriptionPl'])

In [24]:
df_path = f'../data/processed/{topicName}/{fileName}.csv'
df_path_desc =f'../data/processed/{topicName}/{fileName}-desc.csv'

In [25]:
df.to_csv(df_path, index=False)
df_desc.to_csv(df_path_desc, index=False)

In [26]:
destination = f"{topicName}/{fileName}.csv"
destination_desc = f"{topicName}/{fileName}-desc.csv"
destination_original = f"{topicName}/{fileName}-source.{sourceFileExt}"

In [27]:
from src.utils.upload_file_gpc import upload_file

upload_file(df_path,destination)
upload_file(df_path,destination_desc)
upload_file(original_data_path,destination_original)

File ../data/processed/driving-under-influence-pl/dui-pl.csv uploaded to legal-charts-datasets/driving-under-influence-pl/dui-pl.csv.
File ../data/processed/driving-under-influence-pl/dui-pl.csv uploaded to legal-charts-datasets/driving-under-influence-pl/dui-pl-desc.csv.
File ../data/raw/driving-under-influence-pl/dui_pl.csv uploaded to legal-charts-datasets/driving-under-influence-pl/dui-pl-source.csv.


In [28]:
df.dtypes

year              int64
numberOfCrimes    int64
dtype: object

In [29]:
db.close()